<a href="https://colab.research.google.com/github/jinwooahnKHU/NLP/blob/main/Named_entity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 개체명 인식 모델

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 557 kB/s 
     |████████████████████████████████| 806 kB 8.3 MB/s 
     |████████████████████████████████| 8.0 MB 1.6 MB/s 
     |████████████████████████████████| 57 kB 6.7 MB/s 
     |████████████████████████████████| 2.8 MB 55.3 MB/s 
     |████████████████████████████████| 398 kB 70.2 MB/s 
     |████████████████████████████████| 829 kB 60.0 MB/s 
     |████████████████████████████████| 136 kB 66.9 MB/s 
     |████████████████████████████████| 636 kB 60.3 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x3a20e000 @  0x7f01c72c1615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |██████████

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
import torch
from ratsnlp.nlpbook.ner import NERTrainArguments

args=NERTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="ner",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed= 7,
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters NERTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='named-entity-recognition', downstream_corpus_name='ner', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


set seed: 7


In [5]:
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 17.9M/17.9M [00:00<00:00, 18.8MB/s]
Downloading: 100%|██████████| 1.13M/1.13M [00:00<00:00, 15.1MB/s]


In [6]:
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [7]:
from ratsnlp.nlpbook.ner import NERCorpus, NERDataset
corpus = NERCorpus(args)
#NER Dataset class 는 corpus을 학습 형태로 가공(NERFeatures). input_ids, attention_mask, token_type_ids, label_ids
train_dataset=NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/ner/train.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 이어 옆으로 움직여 김일성의 오른쪽에서 한 차례씩 두 번 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.
INFO:ratsnlp:target: 이어 옆으로 움직여 <김일성:PER>의 오른쪽에서 <한 차례:NOH>씩 <두 번:NOH> 상체를 굽혀 조문했으며 이윽고 안경을 벗고 손수건으로 눈주위를 닦기도 했다.

INFO:ratsnlp:tokens: [CLS] 이어 옆 ##으로 움직 ##여 김일성 ##의 오른 ##쪽에서 한 차례 ##씩 두 번 상 ##체를 굽 ##혀 조문 ##했 ##으며 이 ##윽 ##고 안 ##경을 벗고 손 ##수 ##건으로 눈 ##주 ##위를 닦 ##기도 했다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O O B-PER O O O B-NOH I-NOH O B-NOH I-NOH O O O O O O O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:features:

In [8]:
train_dataset[3]

NERFeatures(input_ids=[2, 1, 3476, 4153, 2270, 4042, 420, 4185, 4346, 11, 11524, 12, 197, 9250, 11662, 8805, 8217, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label_ids=[0, 4, 5, 15, 4, 4, 5, 15, 15, 4, 6, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [9]:
train_dataset[3].label_ids

[0,
 4,
 5,
 15,
 4,
 4,
 5,
 15,
 15,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [10]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn = nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [11]:
from torch.utils.data import SequentialSampler

val_dataset = NERDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader=DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/ner
INFO:ratsnlp:loading val data... LOOKING AT /content/Korpora/ner/val.txt
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다
INFO:ratsnlp:target: 결국 초연은 <4년 반:DUR>이 지난 후에 <드레스덴:LOC>에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다

INFO:ratsnlp:tokens: [CLS] 결국 초 ##연 ##은 4년 반 ##이 지난 후에 드 ##레스 ##덴 ##에서 연 ##주 ##되었고 재 ##연 ##도 이루어 ##졌지 ##만 , 이후에 그대로 방치 ##되고 말았 ##다 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: [CLS] O O O O B-DUR I-DUR O O O B-LOC I-LOC I-LOC O O O O O O O O O O O O O O O O O [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INF

In [16]:
from transformers import BertConfig, BertForTokenClassification 

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = corpus.num_labels,
)

model = BertForTokenClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the

In [17]:
from ratsnlp.nlpbook.ner import NERTask

task=NERTask(model, args)

In [18]:
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-ner exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [19]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | BertForTokenClassification | 108 M 
-----------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.389   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [21]:
from ratsnlp.nlpbook.ner import NERDeployArguments

args= NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/epoch=1-val_loss=0.20.ckpt
downstream_model_labelmap_fpath: /gdrive/My Drive/nlpbook/checkpoint-ner/label_map.txt


In [22]:
fine_tuned_model_ckpt=torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

In [23]:
from transformers import BertConfig

pretrained_model_config=BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [24]:
from transformers import BertForTokenClassification

model=BertForTokenClassification(pretrained_model_config)

In [25]:
model.load_state_dict({k.replace("model.",""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [26]:
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [27]:
# label map dictionary
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  
  id_to_label[idx] = label

In [28]:
id_to_label

{0: '[CLS]',
 1: '[SEP]',
 2: '[PAD]',
 3: '[MASK]',
 4: 'O',
 5: '인명',
 6: '기타 수량표현',
 7: '기타',
 8: '기관명',
 9: '날짜',
 10: '지명',
 11: '통화',
 12: '비율',
 13: '시간',
 14: '기간',
 15: '인명',
 16: '기타 수량표현',
 17: '기타',
 18: '기관명',
 19: '날짜',
 20: '지명',
 21: '통화',
 22: '비율',
 23: '시간',
 24: '기간'}

In [29]:
#inference 
def inference_fn(sentence):
  inputs = tokenizer(
      [sentence],
      max_length = args.max_seq_length,
      padding="max_length",
      truncation=True,
  )

  with torch.no_grad():
    outputs=model(**{k : torch.tensor(v) for k, v in inputs.items()})
    probs=outputs.logits[0].softmax(dim=1)
    top_probs, preds = torch.topk(probs, dim=1, k = 1) #각 토큰이 속하는 개체명 확률 분포 중 가장 높은 확률 값과, 개체명 인덱스(preds)구하기 
    tokens=tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_tags = [id_to_label[pred.items()] for pred in preds]
    result=[]
    for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
      if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
        token_result = {
            "token" : token,
            "predicted_tag" : predicted_tag,
            "top_prob" : str(round(top_prob[0].item(),4)),
        }
        result.append(token_result)
  
  return {
      "sentence" : sentence,
      "result" : result,
  }

In [30]:
from ratsnlp.nlpbook.ner import get_web_service_app

app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.ner.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c52d-34-132-194-239.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
